In [1]:
import numpy as np
import pickle
import os

from os import listdir
from sklearn.preprocessing import LabelBinarizer
from keras.models import Sequential
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation, Flatten, Dropout, Dense
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from keras.preprocessing import image
from keras.preprocessing.image import img_to_array
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.utils import Bunch
import matplotlib.pyplot as plt
from pathlib import Path
from skimage.io import imread
from skimage.transform import resize
from keras.applications.mobilenet import MobileNet
from keras.optimizers import SGD
from keras.layers import Dense, GlobalAveragePooling2D
import os.path
import fnmatch
import itertools
import functools
from keras.models import Model
import tensorflow as tf

Using TensorFlow backend.


In [2]:
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

In [3]:
# Check GPU is using or not
import tensorflow as tf
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
sess =  tf.compat.v1.Session(config=config)

In [4]:
train_data_dir = 'data/train'
validation_data_dir = 'data/val'

In [5]:
train_datagen = ImageDataGenerator()
validation_datagen = ImageDataGenerator()
test_datagen = ImageDataGenerator()

In [6]:
EPOCHS = 100
INIT_LR = 1e-3
BS = 8
default_image_size = tuple((224, 224))
image_size = 0
width=299
height=299
depth=3

In [7]:
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(height, width),
    batch_size=BS,
    class_mode='categorical')

validation_generator = validation_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(height, width),
    batch_size=BS,
    class_mode='categorical')
test_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(height, width),
    batch_size=BS,
    class_mode='categorical',
    shuffle = False)

Found 2800 images belonging to 7 classes.
Found 700 images belonging to 7 classes.
Found 700 images belonging to 7 classes.


In [8]:
base_model = MobileNet(weights=None, include_top=False)
# add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)
# let's add a fully-connected layer
x = Dense(1024, activation='relu')(x)
# and a logistic layer -- let's say we have 200 classes
predictions = Dense(7, activation='softmax')(x)
# this is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)

In [9]:
for layer in base_model.layers:
    layer.trainable = True

In [10]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

In [11]:
from tensorflow.keras.callbacks import ModelCheckpoint

checkpoint = ModelCheckpoint('best_model.h5', verbose=1, monitor='val_accuracy', save_best_only=True, mode='auto')

In [12]:
history = model.fit_generator(
    train_generator,
    validation_data=validation_generator,
    steps_per_epoch=train_generator.n // BS,
    validation_steps=validation_generator.n // BS,
    epochs=EPOCHS, verbose=1,
    callbacks=[checkpoint],
)

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/100
350/350 [==============================] - ETA: 0s - loss: 1.8948 - accuracy: 0.2618
Epoch 00001: val_accuracy improved from -inf to 0.14224, saving model to best_model.h5
350/350 [==============================] - 54s 153ms/step - loss: 1.8948 - accuracy: 0.2618 - val_loss: 2.2010 - val_accuracy: 0.1422
Epoch 2/100
350/350 [==============================] - ETA: 0s - loss: 1.5304 - accuracy: 0.3829
Epoch 00002: val_accuracy improved from 0.14224 to 0.32471, saving model to best_model.h5
350/350 [==============================] - 54s 154ms/step - loss: 1.5304 - accuracy: 0.3829 - val_loss: 2.5124 - val_accuracy: 0.3247
Epoch 3/100
350/350 [==============================] - ETA: 0s - loss: 1.3770 - accuracy: 0.4496
Epoch 00003: val_accuracy improved from 0.32471 to 0.39224, saving model to best_model.h5
350/350 [==============================] - 54s 153ms/step - loss: 1.3770 - accuracy: 0.4496 - val_

350/350 [==============================] - ETA: 0s - loss: 0.1831 - accuracy: 0.9521
Epoch 00027: val_accuracy did not improve from 0.88793
350/350 [==============================] - 53s 151ms/step - loss: 0.1831 - accuracy: 0.9521 - val_loss: 1.8571 - val_accuracy: 0.6983
Epoch 28/100
350/350 [==============================] - ETA: 0s - loss: 0.1670 - accuracy: 0.9575
Epoch 00028: val_accuracy improved from 0.88793 to 0.89655, saving model to best_model.h5
350/350 [==============================] - 52s 150ms/step - loss: 0.1670 - accuracy: 0.9575 - val_loss: 0.5846 - val_accuracy: 0.8966
Epoch 29/100
350/350 [==============================] - ETA: 0s - loss: 0.1589 - accuracy: 0.9525
Epoch 00029: val_accuracy did not improve from 0.89655
350/350 [==============================] - 52s 148ms/step - loss: 0.1589 - accuracy: 0.9525 - val_loss: 0.5571 - val_accuracy: 0.8721
Epoch 30/100
350/350 [==============================] - ETA: 0s - loss: 0.1558 - accuracy: 0.9593
Epoch 00030: val_ac

Epoch 55/100
350/350 [==============================] - ETA: 0s - loss: 0.0615 - accuracy: 0.9843
Epoch 00055: val_accuracy did not improve from 0.92816
350/350 [==============================] - 54s 154ms/step - loss: 0.0615 - accuracy: 0.9843 - val_loss: 0.8253 - val_accuracy: 0.8664
Epoch 56/100
350/350 [==============================] - ETA: 0s - loss: 0.1004 - accuracy: 0.9800
Epoch 00056: val_accuracy did not improve from 0.92816
350/350 [==============================] - 53s 152ms/step - loss: 0.1004 - accuracy: 0.9800 - val_loss: 0.5650 - val_accuracy: 0.9009
Epoch 57/100
350/350 [==============================] - ETA: 0s - loss: 0.1087 - accuracy: 0.9789
Epoch 00057: val_accuracy did not improve from 0.92816
350/350 [==============================] - 53s 150ms/step - loss: 0.1087 - accuracy: 0.9789 - val_loss: 0.5775 - val_accuracy: 0.9109
Epoch 58/100
350/350 [==============================] - ETA: 0s - loss: 0.0806 - accuracy: 0.9843
Epoch 00058: val_accuracy did not improve

Epoch 84/100
350/350 [==============================] - ETA: 0s - loss: 0.0999 - accuracy: 0.9843
Epoch 00084: val_accuracy did not improve from 0.93678
350/350 [==============================] - 53s 151ms/step - loss: 0.0999 - accuracy: 0.9843 - val_loss: 0.5095 - val_accuracy: 0.9339
Epoch 85/100
350/350 [==============================] - ETA: 0s - loss: 0.0588 - accuracy: 0.9893
Epoch 00085: val_accuracy did not improve from 0.93678
350/350 [==============================] - 56s 160ms/step - loss: 0.0588 - accuracy: 0.9893 - val_loss: 0.4136 - val_accuracy: 0.9353
Epoch 86/100
350/350 [==============================] - ETA: 0s - loss: 0.0864 - accuracy: 0.9861
Epoch 00086: val_accuracy did not improve from 0.93678
350/350 [==============================] - 53s 153ms/step - loss: 0.0864 - accuracy: 0.9861 - val_loss: 1.3706 - val_accuracy: 0.8693
Epoch 87/100
350/350 [==============================] - ETA: 0s - loss: 0.0489 - accuracy: 0.9896
Epoch 00087: val_accuracy did not improve

In [13]:
print("[INFO] Calculating model accuracy")
scores = model.evaluate_generator(generator=test_generator, steps=test_generator.n // BS +1)
print(f"Test Accuracy: {scores[1]*100}")

[INFO] Calculating model accuracy
Instructions for updating:
Please use Model.evaluate, which supports generators.
Test Accuracy: 92.28571653366089


In [14]:
from sklearn.metrics import classification_report, confusion_matrix
Y_pred = model.predict_generator(test_generator, steps=test_generator.n // BS + 1)

y_pred = np.argmax(Y_pred, axis=1)
print()

print('Confusion Matrix')
target_names = list(test_generator.class_indices.keys())
print(target_names)
print(confusion_matrix(test_generator.classes, y_pred))
print('Classification Report')

print(classification_report(test_generator.classes, y_pred, target_names=target_names))

Instructions for updating:
Please use Model.predict, which supports generators.

Confusion Matrix
['BulBul', 'Chorui', 'Crow', 'Doel', 'Machranga', 'Shalik', 'cuckoo']
[[86  8  2  1  0  0  3]
 [ 1 88  9  1  0  0  1]
 [ 0  0 84  9  0  0  7]
 [ 0  0  2 90  8  0  0]
 [ 1  0  2  2 88  5  2]
 [ 1  1  6  1  0 81 10]
 [ 8  0  8  0  0  0 84]]
Classification Report
              precision    recall  f1-score   support

      BulBul       0.89      0.86      0.87       100
      Chorui       0.91      0.88      0.89       100
        Crow       0.74      0.84      0.79       100
        Doel       0.87      0.90      0.88       100
   Machranga       0.92      0.88      0.90       100
      Shalik       0.94      0.81      0.87       100
      cuckoo       0.79      0.84      0.81       100

    accuracy                           0.86       700
   macro avg       0.86      0.86      0.86       700
weighted avg       0.86      0.86      0.86       700

